In [ ]:
import numpy as np
import pandas as pd
import json
import os
from sklearn.model_selection import train_valid_split
import matplotlib.pyplot as plt
from sys import path 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
current_path = os.getcwd()

systematics_path = os.path.join(current_path,'ingestion', 'systematics')
path.append(systematics_path)
from systematics import postprocess 
input_dir = os.path.join("D:","Higgs_Uncertainity_challenge_input_data")
train_data_file = os.path.join(input_dir, 'train', 'data', 'data.csv')
train_labels_file = os.path.join(input_dir, 'train', 'labels', "data.labels")
train_settings_file = os.path.join(input_dir, 'train', 'settings', "data.json")
train_weights_file = os.path.join(input_dir, 'train', 'weights', "data.weights")

# read train data
data = pd.read_csv(train_data_file)

# read train labels
with open(train_labels_file, "r") as f:
    labels = np.array(f.read().splitlines(), dtype=float)

# read train settings
with open(train_settings_file) as f:
    settings = json.load(f)

# read train weights
with open(train_weights_file) as f:
    weights = np.array(f.read().splitlines(), dtype=float
                       )
    


In [ ]:
theta = 0.5
gamma_roi = train_weights[train_labels == 1].sum()
beta_roi = train_weights[train_labels == 0].sum()

def _sigma_asimov_CR(mu):
    return 0

def calculate_NLL( mu_scan, weight_data,use_CR=False):

    def _sigma_asimov_SR(mu):
        return mu*gamma_roi + beta_roi

    sum_data_total_SR = weight_data.sum()
    sum_data_total_CR = 0
    comb_llr = []
    for i, mu in enumerate(mu_scan):
        hist_llr = (
            -2
            * sum_data_total_SR
            * np.log((_sigma_asimov_SR(mu) / _sigma_asimov_SR(1.0)))
        ) + (2 * (_sigma_asimov_SR(mu) - _sigma_asimov_SR(1.0)))

        if use_CR:
            hist_llr_CR = (
                -2
                * sum_data_total_CR
                * np.log((_sigma_asimov_CR(mu) / _sigma_asimov_CR(1.0)))
                ) + (2 * (_sigma_asimov_CR(mu) - _sigma_asimov_CR(1.0)))
        else:
            hist_llr_CR=0
            #print("do not use CR")    
        #DR time is spent here ?        


        comb_llr.append(hist_llr + hist_llr_CR)

    comb_llr = np.array(comb_llr)
    comb_llr = comb_llr - np.amin(comb_llr)

    return comb_llr



In [ ]:

# Train valid split

X_train ,X_holdout, y_train, y_holdout, w_train, w_holdout = train_valid_split(data, labels, weights, valid_size=0.5)

X_train ,X_valid, y_train, y_valid, w_train, w_valid = train_valid_split(X_train, y_train, w_train, valid_size=0.1)

total_signal = np.sum(weights[labels == 1])
total_signal_train = np.sum(w_train[y_train == 1])
total_signal_valid = np.sum(w_valid[y_valid == 1])
total_signal_holdout = np.sum(w_holdout[y_holdout == 1])
w_train[y_train == 1] *= total_signal / total_signal_train
w_valid[y_valid == 1] *= total_signal / total_signal_valid
w_holdout[y_holdout == 1] *= total_signal / total_signal_holdout

total_background = np.sum(weights[labels == 0])
total_background_train = np.sum(w_train[y_train == 0])
total_background_valid = np.sum(w_valid[y_valid == 0])
total_background_holdout = np.sum(w_holdout[y_holdout == 0])
w_train[y_train == 0] *= total_background / total_background_train
w_valid[y_valid == 0] *= total_background / total_background_valid
w_holdout[y_holdout == 0] *= total_background / total_background_holdout


train_data = X_train.copy()
train_data['labels'] = y_train
train_data['weights'] = w_train

train_data = postprocess(train_data)

y_train = train_data.pop('labels')
w_train = train_data.pop('weights')
X_train = train_data.copy()

del train_data

holdout_data = X_holdout.copy()
holdout_data['labels'] = y_holdout
holdout_data['weights'] = w_holdout

holdout_data = postprocess(holdout_data)

y_holdout = holdout_data.pop('labels')
w_holdout = holdout_data.pop('weights')
X_holdout = holdout_data.copy()

del holdout_data




class_weights_train = (w_train[y_train == 0].sum(), w_train[y_train == 1].sum())

for i in range(len(class_weights_train)):  # loop on B then S target
    # training dataset: equalize number of background and signal
    w_train[y_train == i] *= max(class_weights_train) / class_weights_train[i]
    # valid dataset : increase test weight to compensate for sampling

print("[*] --- Training Model")
X_train_sc = scaler.fit_transform(X_train)

from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=100, n_jobs=4)
model.fit(X_train_sc, y_train, sample_weight=w_train)

y_pred_train = model.predict_proba(X_train_sc)

from sklearn.metrics import roc_auc_score
print("[*] --- Training ROC AUC Score: ", roc_auc_score(y_train, y_pred_train[:, 1], sample_weight=w_train))


In [ ]:
val_min = 1
theta_list = np.linspace(0.8, 1,100)
for theta in theta_list:

    print("theta: ", theta)

    # predict probabilities for holdout
    X_holdout_sc = scaler.transform(X_holdout)
    y_pred = model.predict_proba(X_holdout_sc)
    y_pred = y_pred[:,1]

    y_pred = (y_pred>theta).astype(int)
    gamma_roi = (w_holdout*(y_pred * y_holdout)).sum()
    beta_roi = (w_holdout*(y_pred * (1-y_holdout))).sum()

    valid_data = X_valid.copy()
    valid_data['labels'] = y_valid
    valid_data['weights'] = w_valid

    valid_data_ps = postprocess(valid_data)

    valid_label = valid_data_ps.pop('labels')
    valid_weights = valid_data_ps.pop('weights')


    X_valid_sc = scaler.fit_transform(valid_data_ps)

    X_valid_sc = scaler.fit_transform(X_valid)

    y_pred = model.predict_proba(X_valid_sc) 

    y_pred = y_pred[:,1]
    y_pred = (y_pred>theta).astype(int)

    weight = w_valid*(y_pred)

    mu_scan = np.linspace(0, 3, 100)
    hist_llr = calculate_NLL(mu_scan, weight, use_CR=False)
    hist_llr = np.array(hist_llr)

    val =  np.abs(mu_scan[np.argmin(hist_llr)] - 1)

    if val < val_min:
        val_min = val
        print("val: ", val)
        print("gamma_roi: ", gamma_roi)
        print("beta_roi: ", beta_roi)
        print("Uncertainity", np.sqrt(gamma_roi + beta_roi)/gamma_roi)
        Beta_roi = beta_roi
        Gamma_roi = gamma_roi

        best_theta = theta
        hist_llr_best = hist_llr


In [ ]:
print("best_theta: ", best_theta)
print("val_min: ", val_min)
print(Beta_roi)
print(Gamma_roi)
del beta_roi
del gamma_roi
beta_roi = Beta_roi
gamma_roi = Gamma_roi

In [ ]:




print("min NLL: ", min(hist_llr_best))
print("min NLL index: ", np.argmin(hist_llr_best))
print("min NLL mu: ", mu_scan[np.argmin(hist_llr_best)])


sigma_7 = (max(mu_scan[np.where((hist_llr_best <= 49) & (hist_llr_best >= 36))]), min(mu_scan[np.where((hist_llr_best < 49) & (hist_llr_best > 36))]))
sigma_6 = (max(mu_scan[np.where((hist_llr_best <= 36) & (hist_llr_best >= 25))]), min(mu_scan[np.where((hist_llr_best < 36) & (hist_llr_best > 25))]))
sigma_5 = (max(mu_scan[np.where((hist_llr_best <= 25) & (hist_llr_best >= 16))]), min(mu_scan[np.where((hist_llr_best < 25) & (hist_llr_best > 16))]))
sigma_4 = (max(mu_scan[np.where((hist_llr_best <= 16) & (hist_llr_best >= 9))]), min(mu_scan[np.where((hist_llr_best < 16) & (hist_llr_best > 9))]))
sigma_3 = (max(mu_scan[np.where((hist_llr_best <= 9) & (hist_llr_best >= 4))]), min(mu_scan[np.where((hist_llr_best < 9) & (hist_llr_best > 4))]))
sigma_2 = (max(mu_scan[np.where((hist_llr_best <= 4) & (hist_llr_best >= 1))]), min(mu_scan[np.where((hist_llr_best < 4) & (hist_llr_best > 1))]))
sigma_1 = (max(mu_scan[np.where((hist_llr_best <= 1) & (hist_llr_best >= 0))]), min(mu_scan[np.where((hist_llr_best < 1) & (hist_llr_best > 0))]))


plt.plot( mu_scan,hist_llr_best)
plt.hlines(1, 0, 3,  colors="r", linestyles="dashed", label=f"1 sigma: {sigma_1[0]:.3f} {sigma_1[1]:.3f}")
plt.hlines(4, 0, 3,  colors="b", linestyles="dashed", label=f"2 sigma: {sigma_2[0]:.3f} {sigma_2[1]:.3f}")
plt.hlines(9, 0, 3,  colors="g", linestyles="dashed", label=f"3 sigma: {sigma_3[0]:.3f} {sigma_3[1]:.3f}")
plt.hlines(16, 0, 3, colors="y", linestyles="dashed", label=f"4 sigma: {sigma_4[0]:.3f} {sigma_4[1]:.3f}")
plt.hlines(25, 0, 3, colors="c", linestyles="dashed", label=f"5 sigma: {sigma_5[0]:.3f} {sigma_5[1]:.3f}")
plt.hlines(36, 0, 3, colors="m", linestyles="dashed", label=f"6 sigma: {sigma_6[0]:.3f} {sigma_6[1]:.3f}")
plt.hlines(49, 0, 3, colors="k", linestyles="dashed", label=f"7 sigma: {sigma_7[0]:.3f} {sigma_7[1]:.3f}")
plt.legend()

plt.xlabel("mu")    
plt.ylabel("NLL")
plt.grid(True)
plt.show()



In [ ]:
def shuffle_dataframe(df):
    # Convert DataFrame to NumPy array
    array = df.to_numpy()

    # Shuffle the array
    np.random.shuffle(array)

    # Convert the shuffled array back to a DataFrame
    shuffled_df = pd.DataFrame(array, columns=df.columns)

    return shuffled_df


In [ ]:
p16s = []
p84s = []
test_data_file = os.path.join(input_dir, 'test', 'data', f'data.csv')

for i in range(100):



    signal_valid_bs = w_valid_bs[y_valid_bs == 1]
    background_valid_bs = w_valid_bs[y_valid_bs == 0]
    print("signal_valid_bs: ", signal_valid_bs.sum())
    print("background_valid_bs: ", background_valid_bs.sum())



    X_valid_bs = scaler.fit_transform(test_df)

    y_pred = model.predict_proba(X_valid_bs)
    y_pred = y_pred[:,1]

    y_pred = (y_pred>best_theta).astype(int)


    weight = w_valid_bs*(y_pred)

    print("weight: ", weight.sum())
    print("mu: ", (weight.sum() - beta_roi)/gamma_roi)
    print("gamma_roi: ", gamma_roi)
    print("beta_roi: ", beta_roi)


    mu_scan = np.linspace(0, 3, 100)
    hist_llr = calculate_NLL(mu_scan, weight, use_CR=False)
    hist_llr = np.array(hist_llr)

    if (mu_scan[np.where((hist_llr <= 1.0) & (hist_llr >= 0.0))].size == 0):
        p16 = 0
        p84 = 0
    else:
        p16 = min(mu_scan[np.where((hist_llr <= 1.0) & (hist_llr >= 0.0))])
        p84 = max(mu_scan[np.where((hist_llr <= 1.0) & (hist_llr >= 0.0))]) 

    p16s.append(p16)
    p84s.append(p84)

    plt.plot( mu_scan,hist_llr)

plt.hlines(1, 0, 3, colors="r", linestyles="dashed")
plt.hlines(4, 0, 3, colors="b", linestyles="dashed")
plt.hlines(9, 0, 3, colors="y", linestyles="dashed")


In [ ]:
for i, (p16, p84) in enumerate(zip(p16s, p84s)):
    plt.hlines(y=i, xmin=p16, xmax=p84, colors='b')



print("p16s: ", p16s)
print("p84s: ", p84s)

In [ ]:
def Quantiles_Score( mu, p16, p84, eps=1e-3):
    
    def Interval(p16, p84):
        """Compute the average of the intervals defined by vectors p16 and p84."""
        return np.mean(p84 - p16)

    def Coverage(mu, p16, p84):
        """Compute the fraction of times scalar mu is within intervals defined by vectors p16 and p84."""
        return_coverage = np.mean((mu >= p16) & (mu <= p84))
        return return_coverage

    def f(x, n_tries, max_coverage=1e4, one_sigma = 0.6827):
            sigma68 = np.sqrt(((1-one_sigma)*one_sigma*n_tries))/n_tries

            if (x >= one_sigma-2*sigma68 and x <= one_sigma+2*sigma68):
                out = 1
            elif (x < one_sigma-2*sigma68):
                out = 1 + abs((x-(one_sigma-2*sigma68))/sigma68)**4
            elif (x > one_sigma+2*sigma68):
                out = 1 + abs((x-(one_sigma+2*sigma68))/sigma68)**3
            return out


    coverage = Coverage(mu, p16, p84)
    interval = Interval(p16, p84)
    score = -np.log((interval + eps) * f(coverage, n_tries=mu.shape[0]))
    return interval, coverage, score


In [ ]:
mus = np.ones(100)
p16s = np.array(p16s)
p84s = np.array(p84s)

print("Quantiles_Score: ", Quantiles_Score(mus, p16s, p84s))